In [0]:
# required imports
import sys 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline  


import torch
import torchvision 
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
# mount the google drive to download the datasets
from google.colab import drive
drive.mount('/content/drive')
#project_path = 'include pathe to where your notebook is located on the drive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# create transforms to load the images, nothing much is needed here. 

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Normalize the test set same as training set without augmentation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


In [4]:
# download CIFAR 10 training set
trainset = torchvision.datasets.CIFAR10('../data', train=True,
                                        download=True, transform=transform_train)

# load the trainning set
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)

# download the test data
testset = torchvision.datasets.CIFAR10('../data', train=True,
                                        download=True, transform=transform_test)

# load the test data
testloader = torch.utils.data.DataLoader(
    testset, batch_size=4, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# check those manually on the dataset site: https://www.cs.toronto.edu/~kriz/cifar.html 

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# divide the training dataset into the required groups Make sure they are balanced
# original trainset is made of 50k images

total_size = len(trainset)
test_size = len(testset)
#loader_size = len(trainloader)
split1 = total_size // 4
split2 = split1 * 2
split3 = split1 * 3

#print(loader_size)

print(total_size, test_size, split1, split2, split3)

indices = list(range(total_size))

# two groups to train the shadow (in and out)
shadow_train_idx = indices[:split1]
shadow_out_idx = indices[split1:split2]

# two groups to train the Target (in and out)
target_train_idx = indices[split2:split3]
target_out_idx = indices[split3:total_size]

50000 50000 12500 25000 37500


In [0]:
train_batch_size = 128 # pick your own
test_batch_size = 64

#train_batch_size = 128 # maybe only need one
#test_batch_size = 64

# divide and load shadow train in and out
shadow_train_sampler = SubsetRandomSampler(shadow_train_idx) # Pytorch function
shadow_train_loader = torch.utils.data.DataLoader(trainset, batch_size=train_batch_size, sampler=shadow_train_sampler)

shadow_out_sampler = SubsetRandomSampler(shadow_out_idx)
shadow_out_loader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size, sampler=shadow_out_sampler)

# divide and load Target in and out
target_train_sampler = SubsetRandomSampler(target_train_idx)
target_out_sampler = SubsetRandomSampler(target_out_idx)

target_train_loader = torch.utils.data.DataLoader(trainset, batch_size=train_batch_size, sampler=target_train_sampler)

target_out_loader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size, sampler=target_out_sampler)

In [0]:
# create a CNN
# Input shape (3, 32, 32) 
# architecture: simple. 2 conv and 2 Max pool, followed by 2 fc (120, 84) 
# output of fc is 10 because we have 10 classes!



class CNN(nn.Module):
    """CNN."""

    def __init__(self):
        """CNN Builder."""
        super(CNN, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

        )


        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(8192, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 10)
        )


    def forward(self, x):
        """Perform forward."""
        
        # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)

        return x

In [8]:
# initalize a target model and train it
epochs = 20
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

target_model = CNN()
target_model.to(device)
criterion = nn.CrossEntropyLoss() # CrossEntropyLoss
optimizer = optim.Adam(target_model.parameters(), lr = .001)# try Adam VS SGD

# let the magic begin
for epoch in range(epochs):  

    running_loss = 0.0
    
    for i, data in enumerate(target_train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        # make a prediction: forward prop
        #print(inputs.shape)
        outputs = target_model(inputs)
                       
        # calculate loss
        loss = criterion(outputs, labels)
                       
        # backwards propogation
        loss.backward()       
        
       

        # calculate gradients
        optimizer.step()
        # updaate weights in backprop
                       
                       
        running_loss += loss.data
        # print statistics
        
    ##train_accuracy = calculate_accuracy(trainloader)
    ##test_accuracy = calculate_accuracy(testloader)

    running_loss_normal = running_loss / len(target_train_loader)
    print("Running Loss: %f , Normalized: %f" % (running_loss, running_loss_normal))

    #print("Iteration: {0} | Loss: {1} | Training accuracy: {2}% | Test accuracy: {3}%".format(epoch+1, running_loss, train_accuracy, test_accuracy))
        
        

print('Finished Training the Target model...')

Running Loss: 179.544083 , Normalized: 1.832082
Running Loss: 149.732651 , Normalized: 1.527884
Running Loss: 138.663406 , Normalized: 1.414933
Running Loss: 127.310799 , Normalized: 1.299090
Running Loss: 119.414314 , Normalized: 1.218513
Running Loss: 112.864502 , Normalized: 1.151679
Running Loss: 107.111084 , Normalized: 1.092970
Running Loss: 103.676498 , Normalized: 1.057923
Running Loss: 97.955627 , Normalized: 0.999547
Running Loss: 95.398926 , Normalized: 0.973458
Running Loss: 93.503601 , Normalized: 0.954118
Running Loss: 90.151497 , Normalized: 0.919913
Running Loss: 87.852188 , Normalized: 0.896451
Running Loss: 83.787766 , Normalized: 0.854977
Running Loss: 83.275620 , Normalized: 0.849751
Running Loss: 79.758278 , Normalized: 0.813860
Running Loss: 79.212059 , Normalized: 0.808286
Running Loss: 78.404701 , Normalized: 0.800048
Running Loss: 75.982094 , Normalized: 0.775327
Running Loss: 74.323570 , Normalized: 0.758404
Finished Training the Target model...


In [0]:
## OR load the model from ICP6


In [9]:
# calculate the accuracy of the Target Model
correct = 0
total = 0

with torch.no_grad():
    for data in target_out_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = target_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

##print(total)
print('Accuracy of the network on the 12500 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 12500 test images: 68 %


In [10]:
# TODO: Create shadow model(you aren't supposed to know the parameters of the target network)

shadow_model = nn.Sequential(nn.Linear(3072, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64,10),
                      nn.LogSoftmax(dim=1))

shadow_model.to(device)

Sequential(
  (0): Linear(in_features=3072, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=10, bias=True)
  (5): LogSoftmax()
)

In [11]:
#epochs = 5

# TODO: Train shadow model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(shadow_model.parameters(), lr = .001)

# let the magic begin
for epoch in range(epochs):  

    running_loss = 0.0
    
    for i, data in enumerate(shadow_train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.view(inputs.shape[0], -1)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        # make a prediction: forward prop
        #print(inputs.shape)
        outputs = shadow_model(inputs)
                       
        # calculate loss
        loss = criterion(outputs, labels)
                       
        # backwards propogation
        loss.backward()       
        
       

        # calculate gradients
        optimizer.step()
        # updaate weights in backprop
                       
                       
        running_loss += loss.data
        # print statistics
        
    ##train_accuracy = calculate_accuracy(trainloader)
    ##test_accuracy = calculate_accuracy(testloader)
    running_loss_normal = running_loss / len(target_train_loader)
    print("Running Loss: %f , Normalized: %f" % (running_loss, running_loss_normal))

    #print("Iteration: {0} | Loss: {1} | Training accuracy: {2}% | Test accuracy: {3}%".format(epoch+1, running_loss, train_accuracy, test_accuracy))
        
        


print('Finished Training the Shadow model')

Running Loss: 192.613922 , Normalized: 1.965448
Running Loss: 177.780884 , Normalized: 1.814091
Running Loss: 172.272537 , Normalized: 1.757883
Running Loss: 167.879471 , Normalized: 1.713056
Running Loss: 165.146027 , Normalized: 1.685163
Running Loss: 162.393646 , Normalized: 1.657078
Running Loss: 159.695450 , Normalized: 1.629545
Running Loss: 157.502426 , Normalized: 1.607168
Running Loss: 155.151932 , Normalized: 1.583183
Running Loss: 154.765610 , Normalized: 1.579241
Running Loss: 152.252182 , Normalized: 1.553594
Running Loss: 150.905502 , Normalized: 1.539852
Running Loss: 150.797546 , Normalized: 1.538750
Running Loss: 149.363312 , Normalized: 1.524115
Running Loss: 148.992599 , Normalized: 1.520333
Running Loss: 147.836426 , Normalized: 1.508535
Running Loss: 147.144455 , Normalized: 1.501474
Running Loss: 146.327271 , Normalized: 1.493135
Running Loss: 145.439560 , Normalized: 1.484077
Running Loss: 144.213730 , Normalized: 1.471569
Finished Training the Shadow model


In [12]:
# Test the accuracy of the shadow model
correct = 0
total = 0

#attack_data_set = []

with torch.no_grad():
    for data in shadow_out_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.view(inputs.shape[0], -1)
        outputs = shadow_model(inputs)
        
        #attack_data_set.append((outputs, torch.tensor([1.0], requires_grad=True)))
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        
    print('Accuracy of the shadow network on the shadow_train_loader: %d %%' % (100 * correct / total))
 

Accuracy of the shadow network on the shadow_train_loader: 44 %


In [0]:
# Make preds with shadow_model and create training data for attack model

shadow_model.eval()

for param in shadow_model.parameters():
  param.requires_grad = False

# make predictions on both datasets (shadow_in and shdow_out)
attack_data = []

for inputs, labels in shadow_train_loader:
  inputs, labels = inputs.to(device), labels.to(device)
  inputs = inputs.view(inputs.shape[0], -1)
  predictions = shadow_model(inputs)
  
  for i in range(predictions.shape[0]):
    attack_data.append((predictions[i], 1))
  
for inputs, labels in shadow_out_loader:
  inputs, labels = inputs.to(device), labels.to(device)
  inputs = inputs.view(inputs.shape[0], -1)
  predictions = shadow_model(inputs)
  
  for i in range(predictions.shape[0]):
    attack_data.append((predictions[i], 0))

In [0]:
attack_data_loader = torch.utils.data.DataLoader(attack_data, batch_size=4, shuffle=True)

In [15]:
# create the Attack Model: A NN binary classifier {0, 1}
# the input to this model is the propability distribution vector of size 10
# and the output is either 0 (input was not included in training) or 1



epochs = 5;
# Build a feed-forward network
attack_model = nn.Sequential(nn.Linear(10, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 2),
                      nn.LogSoftmax(dim=1))

attack_model.to(device)

# Define the loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(attack_model.parameters(), lr=.001)


for epoch in range(epochs):  

    running_loss = 0.0
    count = 0
    for i, data in enumerate(attack_data_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        count +=1
        #print("Data loader count:  %d" % count)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        # make a prediction: forward prop
        #print(inputs.shape)
        outputs = attack_model(inputs)
                       
        # calculate loss
        ##print(outputs)
        ##print(labels)
        loss = criterion(outputs, labels)
                       
        # backwards propogation
        loss.backward()       
        
       

        # calculate gradients
        optimizer.step()
        # updaate weights in backprop
                       
                       
        running_loss += loss.data
        # print statistics
        
    ##train_accuracy = calculate_accuracy(trainloader)
    ##test_accuracy = calculate_accuracy(testloader)
    running_loss_normal = running_loss / len(target_train_loader)
    print("Running Loss: %f , Normalized: %f" % (running_loss, running_loss_normal))

    #print("Iteration: {0} | Loss: {1} | Training accuracy: {2}% | Test accuracy: {3}%".format(epoch+1, running_loss, train_accuracy, test_accuracy))
        
        


print('Finished Training the Attack Model')

Running Loss: 4131.594238 , Normalized: 42.159122
Running Loss: 4019.495605 , Normalized: 41.015259
Running Loss: 3955.517578 , Normalized: 40.362423
Running Loss: 3911.081543 , Normalized: 39.908993
Running Loss: 3874.998047 , Normalized: 39.540794
Finished Training the Attack Model


In [0]:
## Could load attack model from ICP 6, but technically it was trained in a white-box scenario

In [0]:
# calculate the recall and precision of your attack network using the Target_out and Target_in datasets
# to do so, take a random numer of datapoints, run them throw the target model,
# and then input the output of the target model to your attack network 
# you already know the target_in and target_out samples, so use that info to evaluate the attack model


# Run attack agains the target model
actual_value = []
pred_value = []

attack_model.eval()

for param in attack_model.parameters():
  param.requires_grad = False

for inputs, labels in target_train_loader:
  inputs, labels = inputs.to(device), labels.to(device)
  
  # Get probability output of target model
  output = target_model(inputs)
  
  # Run through attack model
  pred = attack_model(output)
  
  # Pull out top class
  top_p, top_class = pred.topk(1, dim=1)
  
  for i in range(top_class.shape[0]):
    pred_value.append(top_class[0].item())
    actual_value.append(1)
    
for inputs, labels in target_out_loader:
  inputs, labels = inputs.to(device), labels.to(device)
  
  # Get probability output of target model
  output = target_model(inputs)
  
  # Run through attack model
  pred = attack_model(output)
  
  # Pull out top class
  top_p, top_class = pred.topk(1, dim=1)
  
  for i in range(top_class.shape[0]):
    pred_value.append(top_class[0].item())
    actual_value.append(0)

In [0]:
#for i in range(len(actual_value)):
 # print("Actual: {}   Prediction: {}".format(actual_value[i], pred_value[i]))

In [19]:
# Calculate recall and precision
# precison = true positive / true positive + false positive
true_pos = 0
false_neg = 0

total_positive = sum(pred_value)
for i in range(len(actual_value)):
  if (pred_value[i] == 1) and (actual_value[i] == 1):
    true_pos += 1
  elif (pred_value[i] == 0 and actual_value[i] == 1):
    false_neg += 1
    
print('True positive: {} Total Positive: {} Precision: {}'.format(true_pos, total_positive, true_pos / total_positive))
print('Recall: {}', true_pos / (true_pos + false_neg))

True positive: 7680 Total Positive: 16724 Precision: 0.4592202822291318
Recall: {} 0.6144
